In [1]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_text(text):
    # Text Cleaning
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Stop Word Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back to text
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text


In [2]:
import pandas as pd
data=pd.read_csv('incidents.csv')

In [3]:
data['description']=data.description.astype('str')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [4]:
data['preproc']=data['description'].apply(preprocess_text)

In [18]:
data['preproc']

0       Late August 31 2023 crude oil spill occurred P...
1       drug runner sailboat modified operate semisubm...
2       28AUG2023 USCG Sector Puget Sound notified NOA...
3       USCG District 7 established Crisis Action Team...
4       Late Sunday night 27AUG2023 USCG Sector San Fr...
                              ...                        
4430    1500 June 13 1968 World Glory bound Huelva Spa...
4431    March 7 1968 Greek tank vessel General Colocot...
4432    morning March 3 1968 tanker Ocean Eagle ground...
4433    morning March 18 1967 TV Torrey Canyon ran agr...
4434    Tampico Maru henceforth Tampico left Los Angel...
Name: preproc, Length: 4435, dtype: object

In [5]:
data['threat'].fillna('Unknown', inplace=True)

In [6]:
data['threat'].value_counts()

threat
Oil         2529
Unknown     1340
Chemical     292
Other        274
Name: count, dtype: int64

In [7]:
filtered_data = data[(data['threat'] == 'Oil') | (data['threat'] == 'Chemical')]

In [8]:
filtered_data

id   open_date                                               name  \
0     10659  01-09-2023   Crude Oil Spill at Port Manatee in Tampa Bay, FL   
1     10660  01-09-2023                    Grounded Drug-running Submarine   
2     10657  28-08-2023  26-foot Cabin Cruiser Sunk, Bellingham Bay, Be...   
4     10658  27-08-2023         53-foot Motor Yacht Adrift Off Big Sur, CA   
5     10655  24-08-2023  38-foot Fishing Vessel Listing, Funter Bay, An...   
...     ...         ...                                                ...   
4425   6209  10-02-1970  Chevron Main Pass Block 41; 11 miles E of the ...   
4426   6208  04-02-1970                         Arrow; Nova Scotia, Canada   
4427   6207  30-04-1969            Hamilton Trader; Liverpool Bay, England   
4428   6206  28-01-1969  Santa Barbara Well blowout; Santa Barbara, Cal...   
4429   6205  13-12-1968        Witwater; Galeta Island, Canal Zone, Panama   

                                               location        lat  \
0                               St. Petersburg, FL, USA  27.634018   
1                                       Mona Island, PR  18.073605   
2                                        Bellingham, WA  48.723710   
4                                       off Big Sur, CA  36.028000   
5                           Funter Bay, Angoon, AK, USA  58.231056   
...                                                 ...        ...   
4425  11 miles E of the Mississippi River delta, Lou...  29.383300   
4426                                Nova Scotia, Canada  45.466700   
4427                             Liverpool Bay, England  53.500000   
4428                          Santa Barbara, California  34.166700   
4429                  Galeta Island, Canal Zone, Panama   9.583330   

             lon threat       tags             commodity  measure_skim  \
0     -82.565190    Oil        NaN                 Crude           NaN   
1     -67.931855    Oil        NaN                diesel           NaN   
2    -122.509100    Oil        NaN              gasoline           NaN   
4    -121.867333    Oil        NaN                Diesel           NaN   
5    -134.919000    Oil        NaN                Diesel           NaN   
...          ...    ...        ...                   ...           ...   
4425  -88.983300    Oil        NaN             crude oil           NaN   
4426  -61.100000    Oil  Grounding          Bunker C Oil           NaN   
4427    3.333330    Oil  Collision        No. 6 Fuel Oil           NaN   
4428 -119.750000    Oil   Wellhead  California crude oil           NaN   
4429  -80.666700    Oil  Collision      Diesel, Bunker C           NaN   

      measure_shore  measure_bio  measure_disperse  measure_burn  \
0               NaN          NaN               NaN           NaN   
1               NaN          NaN               NaN           NaN   
2               NaN          NaN               NaN           NaN   
4               NaN          NaN               NaN           NaN   
5               NaN          NaN               NaN           NaN   
...             ...          ...               ...           ...   
4425            NaN          NaN               1.0           NaN   
4426            NaN          NaN               1.0           1.0   
4427            NaN          NaN               1.0           NaN   
4428            NaN          NaN               1.0           NaN   
4429            NaN          NaN               NaN           1.0   

      max_ptl_release_gallons  posts  \
0                      3500.0      0   
1                         NaN      0   
2                        35.0      0   
4                       800.0      0   
5                       250.0      0   
...                       ...    ...   
4425                      NaN      9   
4426                3470000.0      7   
4427                 168000.0      6   
4428                4200000.0      8   
4429                 588000.0      6   

                                            

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample preprocessed text data
preprocessed_text = filtered_data.preproc

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed text data
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_text)

# Get the feature names (words) used for the columns in the TF-IDF matrix
feature_names = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a dense array for better readability (optional)
tfidf_matrix_dense = tfidf_matrix.toarray()

# Print the feature names and TF-IDF matrix
print("Feature Names (Words):")
print(feature_names)
print("\nTF-IDF Matrix:")
print(tfidf_matrix_dense)


Feature Names (Words):
['00' '000' '0000' ... 'zug' 'zulu' 'zuma']

TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X = tfidf_matrix_dense  # TF-IDF matrix
y = filtered_data['threat']  # Target labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = classifier.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print a classification report with precision, recall, and F1-score
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.90
              precision    recall  f1-score   support

    Chemical       0.00      0.00      0.00        58
         Oil       0.90      1.00      0.95       507

    accuracy                           0.90       565
   macro avg       0.45      0.50      0.47       565
weighted avg       0.81      0.90      0.85       565



c:\Users\joel maguluri\miniconda3\envs\llm\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\joel maguluri\miniconda3\envs\llm\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\joel maguluri\miniconda3\envs\llm\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
%pip install -U imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [12]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Check the class distribution before oversampling
print("Class distribution before oversampling:", Counter(y))

# Create an instance of SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to generate synthetic samples
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check the class distribution after oversampling
print("Class distribution after oversampling:", Counter(y_resampled))


Class distribution before oversampling: Counter({'Oil': 2529, 'Chemical': 292})
Class distribution after oversampling: Counter({'Oil': 2529, 'Chemical': 2529})


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize and train the classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = classifier.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print a classification report with precision, recall, and F1-score
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.98
              precision    recall  f1-score   support

    Chemical       0.97      1.00      0.98       493
         Oil       1.00      0.97      0.98       519

    accuracy                           0.98      1012
   macro avg       0.98      0.98      0.98      1012
weighted avg       0.98      0.98      0.98      1012

